## Numerical integration (Riemann sum): calculating $\Phi(1) = \frac 1 {\sqrt{2\pi}} \int_{0}^1 e^{-x^2/2} \, dx$
(see, e.g.: https://mathworld.wolfram.com/NormalDistributionFunction.html).

#### CUDA version with two kernels (trapezoid median + sum reducer)

In [1]:
%%file riemann_cuda_double_reduce.cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <device_launch_parameters.h>

#define N 100000000

/* CUDA error wraper */
static void CUDA_ERROR( cudaError_t err) 
{
    if (err != cudaSuccess) {
        printf("CUDA ERROR: %s, exiting\n", cudaGetErrorString(err));
        exit(-1);
    }
}

__global__ void medianTrapezoid(double *a, int n)
{
  int idx = blockIdx.x * blockDim.x + threadIdx.x;
  double x = (double)idx / (double)n;
 
  if(idx < n)
    a[idx] = (exp(-x * x / 2.0) + exp(-(x + 1 / (double)n) * (x + 1 / (double)n) / 2.0)) / 2.0;
}

__global__ void reducerSum(double *a, double *out, int n, int block_size) {
    int idx = threadIdx.x;
    double sum = 0;
    for (int i = idx; i < n; i += block_size)
        sum += a[i];
    extern __shared__ double r[];
    r[idx] = sum;
    __syncthreads();
    for (int size = block_size/2; size>0; size/=2) {
        if (idx<size)
            r[idx] += r[idx+size];
        __syncthreads();
    }
    if (idx == 0)
        *out = r[0];
}

double riemannCUDA(int n)
{
  ///size of the arrays in bytes
  size_t size = n * sizeof(double);

  int block_size = 1024;

  // allocate array on host and device
  double* a_h = (double *)malloc(size);
  double* out_h = (double *)malloc(sizeof(double));
  double* r = (double *)malloc(block_size * sizeof(double));
  double* a_d; cudaMalloc((double **) &a_d, size);
  double* out; cudaMalloc((double **) &out, sizeof(double));

  // do calculation on device
  
  int n_blocks = n/block_size + (n % block_size == 0 ? 0:1);
  printf("CUDA kernel 'medianTrapezoid' launch with %d blocks of %d threads\n", n_blocks, block_size);
  medianTrapezoid <<< n_blocks, block_size >>> (a_d, n);
  int n_blocks2 = 1;
  printf("CUDA kernel 'reducerSum' launch with %d blocks of %d threads\n\n", n_blocks2, block_size);
  reducerSum <<< n_blocks2, block_size, block_size*sizeof(double) >>> (a_d, out, n, block_size);
  
  // copy results from device to host
  cudaMemcpy(out_h, out, sizeof(double), cudaMemcpyDeviceToHost);

  // add up results
  double sum;
  sum = *out_h;
  sum *= (1.0 / sqrt(2.0 * M_PI)) / (double)n;
  
  // clean up
  free(a_h); cudaFree(a_d);
  free(out_h); cudaFree(out);
  cudaFree(r);
  
  return sum;
}


int main(int argc, char** argv){

  /*get info on our GPU, defaulting to first one*/
  cudaDeviceProp prop;
  CUDA_ERROR(cudaGetDeviceProperties(&prop,0));
  printf("Found GPU '%s' with %g GB of global memory, max %d threads per block, and %d multiprocessors\n", 
         prop.name, prop.totalGlobalMem/(1024.0*1024.0*1024.0),
         prop.maxThreadsPerBlock,prop.multiProcessorCount);
 
  /*init CUDA*/
  CUDA_ERROR(cudaSetDevice(0));

  clock_t t1; 
  t1 = clock();

  double sum = riemannCUDA(N);

  t1 = clock() - t1;

  double time_taken1 = ((double)t1)/CLOCKS_PER_SEC; // in seconds

  printf("Riemann sum CUDA (double precision) for N = %d    : %.17g \n", N, sum);
  printf("Total time (measured by CPU)                              : %f s\n", time_taken1);
}

Overwriting riemann_cuda_double_reduce.cu


In [2]:
!PATH=/usr/local/cuda-10.1/bin:${PATH} nvcc -o riemann_cuda_double_reduce riemann_cuda_double_reduce.cu && ./riemann_cuda_double_reduce

Found GPU 'Quadro P400' with 2 GB of global memory, max 1024 threads per block, and 2 multiprocessors
CUDA kernel 'medianTrapezoid' launch with 97657 blocks of 1024 threads
CUDA kernel 'reducerSum' launch with 1 blocks of 1024 threads

Riemann sum CUDA (double precision) for N = 100000000    : 0.34134474606854298 
Total time (measured by CPU)                              : 0.628118 s


#### OpenCL version with two kernels (trapezoid median + sum reducer)

In [3]:
%%file riemann_reduce.cl
__kernel void medianTrapezoid(__global double *a, int n) {
    
    int idx = get_global_id(0);
    double x = (double)idx / (double)n;
 
    if(idx < n)
       a[idx] = (exp(-x * x / 2.0) + exp(-(x + 1 / (double)n) * (x + 1 / (double)n) / 2.0)) / 2.0;
}

__kernel void reducerSum(__global double *a, __global double *out, __local double *r, int n, int block_size)
{
    int idx = get_local_id(0);
    double sum = 0;
    for (int i = idx; i < n; i += block_size)
        sum += a[i];
    r[idx] = sum;
    barrier(CLK_LOCAL_MEM_FENCE);

    for (int size = block_size/2; size>0; size/=2) {
        if (idx<size)
            r[idx] += r[idx+size];
        barrier(CLK_LOCAL_MEM_FENCE);
    }
   
    if (idx == 0)
        *out = r[0];
}

Overwriting riemann_reduce.cl


In [4]:
%%file riemann_opencl_double_reduce.c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <time.h>
#include <CL/cl.h>

#define MAX_SOURCE_SIZE (0x100000)

#define N 100000000

double riemannCL(int n)
{
    //Allocate memory to host variable
    int block_size = 256;
    double *a = (double*)malloc(sizeof(double) * n);
    double *out = (double*)malloc(sizeof(double));
    
    // Load the kernel source code into the array source_str
    FILE *fp;
    char *source_str;
    size_t source_size;

    fp = fopen("riemann_reduce.cl", "r");
    if (!fp) {
        fprintf(stderr, "Failed to load kernel.\n");
        exit(1);
    }
    source_str = (char*)malloc(MAX_SOURCE_SIZE);
    source_size = fread( source_str, 1, MAX_SOURCE_SIZE, fp);
    fclose( fp );

    // Get platform and device information
    cl_platform_id platform_id = NULL;
    cl_device_id device_id = NULL;   
    cl_uint ret_num_devices;
    cl_uint ret_num_platforms;
    cl_int ret = clGetPlatformIDs(1, &platform_id, &ret_num_platforms);
    ret = clGetDeviceIDs( platform_id, CL_DEVICE_TYPE_ALL, 1, 
            &device_id, &ret_num_devices);

    // Create an OpenCL context
    cl_context context = clCreateContext( NULL, 1, &device_id, NULL, NULL, &ret);

    // Create a command queue
    cl_command_queue command_queue = clCreateCommandQueue(context, device_id, 0, &ret);

    // Create memory buffers on the device for each vector 
    cl_mem a_mem_obj = clCreateBuffer(context, CL_MEM_READ_WRITE, 
            n * sizeof(double), NULL, &ret);
    cl_mem out_mem_obj = clCreateBuffer(context, CL_MEM_READ_WRITE, 
            sizeof(double), NULL, &ret);

    // Create a program from the kernel source
    cl_program program = clCreateProgramWithSource(context, 1, 
            (const char **)&source_str, (const size_t *)&source_size, &ret);

    // Build the program
    ret = clBuildProgram(program, 1, &device_id, NULL, NULL, NULL);

    clock_t t2; 
    t2 = clock(); 

    // Create the OpenCL kernel
    cl_kernel kernel = clCreateKernel(program, "medianTrapezoid", &ret);

    // Set the arguments of the kernel
    ret = clSetKernelArg(kernel, 0, sizeof(cl_mem), (void *)&a_mem_obj);    
    ret = clSetKernelArg(kernel, 1, sizeof(cl_int), (void *)&n);
    
    // Execute the OpenCL kernel
    size_t local_item_size = block_size;
    int n_blocks = n/local_item_size + (n % local_item_size == 0 ? 0:1);
    size_t global_item_size = n_blocks * local_item_size;
    printf("OpenCL kernel 'medianTrapezoid' launch with %d blocks of %lu threads\n", n_blocks, local_item_size);

    ret = clEnqueueNDRangeKernel(command_queue, kernel, 1, NULL, 
            &global_item_size, &local_item_size, 0, NULL, NULL);

    // Create the OpenCL kernel2
    cl_kernel kernel2 = clCreateKernel(program, "reducerSum", &ret);

    // Set the arguments of the kernel2
    ret = clSetKernelArg(kernel2, 0, sizeof(cl_mem), (void *)&a_mem_obj);    
    ret = clSetKernelArg(kernel2, 1, sizeof(cl_mem), (void *)&out_mem_obj);   
    ret = clSetKernelArg(kernel2, 2, block_size * sizeof(cl_double), NULL);    
    ret = clSetKernelArg(kernel2, 3, sizeof(cl_int), (void *)&n);   
    ret = clSetKernelArg(kernel2, 4, sizeof(cl_int), (void *)&block_size);

    // Execute the OpenCL kernel2
    size_t local_item_size2 = block_size;
    size_t global_item_size2 = block_size;
    printf("OpenCL kernel 'reducerSum' launch with %lu blocks of %lu threads\n\n", global_item_size2/local_item_size2, local_item_size2);

    ret = clEnqueueNDRangeKernel(command_queue, kernel2, 1, NULL, 
            &global_item_size2, &local_item_size2, 0, NULL, NULL);

    t2 = clock() - t2;

    double time_taken2 = ((double)t2)/CLOCKS_PER_SEC; // in seconds

    clock_t t3; 
    t3 = clock();

   
    ret = clEnqueueReadBuffer(command_queue, out_mem_obj, CL_TRUE, 0, 
            sizeof(double), out, 0, NULL, NULL);

    t3 = clock() - t3;

    double time_taken3 = ((double)t3)/CLOCKS_PER_SEC; // in seconds

    // add up results
    double sum;
    sum = *out;
    sum *= (1.0 / sqrt(2.0 * M_PI)) / (double)n;

    // Clean up
    ret = clFlush(command_queue);
    ret = clFinish(command_queue);
    ret = clReleaseKernel(kernel);
    ret = clReleaseKernel(kernel2);
    ret = clReleaseProgram(program);
    ret = clReleaseMemObject(a_mem_obj);
    ret = clReleaseMemObject(out_mem_obj);
    ret = clReleaseCommandQueue(command_queue);
    ret = clReleaseContext(context);
    free(a);

    printf("OpenCL and CPU code diagnostics:\n");
    printf("OpenCL kernels execution time (measured by CPU):        %f ms\n", time_taken2 * 1000);
    printf("Device to host memory transfer time (measured by CPU):  %f s\n\n", time_taken3);
  
    return sum;
}

int main(int argc, char** argv){

  clock_t t1; 
  t1 = clock(); 

  double sum = riemannCL(N);

  t1 = clock() - t1;

  double time_taken1 = ((double)t1)/CLOCKS_PER_SEC; // in seconds

  printf("Riemann sum OpenCL (double precision) for N = %d    : %.17g \n", N, sum);
  printf("Total time (measured by CPU)                                : %f s\n", time_taken1);
}

Overwriting riemann_opencl_double_reduce.c


In [5]:
!nvcc -o riemann_opencl_double_reduce riemann_opencl_double_reduce.c -lOpenCL && ./riemann_opencl_double_reduce

OpenCL kernel 'medianTrapezoid' launch with 390625 blocks of 256 threads
OpenCL kernel 'reducerSum' launch with 1 blocks of 256 threads

OpenCL and CPU code diagnostics:
OpenCL kernels execution time (measured by CPU):        0.477000 ms
Device to host memory transfer time (measured by CPU):  0.365489 s

Riemann sum OpenCL (double precision) for N = 100000000    : 0.34134474606854376 
Total time (measured by CPU)                                : 0.623706 s
